In [1]:
from helpers.iterative import IT_DATAMODULE
import random
# random.seed(42)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch import optim
from torch import mps
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer, minmax_scale, power_transform, scale
from tqdm.auto import tqdm
# from tqdm import tqdm
import os

import lightning.pytorch as pl

from helpers.iterative import IT_DATAMODULE
from models.iterative import LSTM, CNN_1D_LSTM

# Set global seed for reproducibility in numpy, torch, scikit-learn

# torch.manual_seed(42)
# torch.mps.manual_seed(42)
# torch.backends.mps.deterministic = True
# torch.cuda.manual_seed(42)
# torch.backends.cudnn.deterministic = True
# np.random.seed(42)
%autosave 0

Autosave disabled


In [18]:
with open("./DATA/Tickers/month_tickers_clean.txt", "r") as f:
    tickers = f.read().strip().split("\n")

In [43]:
# Choose random ticker from tickers
ticker = random.choice(range(len(tickers)))
ticker

308

In [2]:
LEARING_RATE = 1e-4 # 1e-4 ind standard
EPOCHS = 300
BATCH_SIZE = 32
PRED_HORIZON = 1
LOOKBACK = 12
pl.seed_everything(42)

Global seed set to 42


42

In [3]:
fin_data = pd.read_parquet("./DATA/Monthly/Processed/month_data_fin_tec.parquet")

In [62]:
fin_data.filter(regex="^RGD_").pct_change()*100

,RGD_CR,RGD_PER,RGD_DER,RGD_OP,RGD_FCF,RGD_ROE,RGD_HP,RGD_PBR,RGD_LP,RGD_PM,...,RGD_momentum_ppo,RGD_momentum_ppo_signal,RGD_momentum_ppo_hist,RGD_momentum_pvo,RGD_momentum_pvo_signal,RGD_momentum_pvo_hist,RGD_momentum_kama,RGD_others_dr,RGD_others_dlr,RGD_others_cr
2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,NaN,NaN,NaN,0.000000,NaN,NaN,122.222222,NaN,-14.285714,NaN,...,inf,inf,inf,inf,inf,inf,84.519787,-22.613251,inf,inf
2000-03-31,NaN,NaN,NaN,242.857143,NaN,NaN,20.000000,NaN,100.000000,NaN,...,17.840281,97.840281,-2.159719,75.412390,155.412390,55.412390,0.073369,-118.846154,-141.033896,-53.846154
2000-04-30,NaN,NaN,NaN,-45.833333,NaN,NaN,-45.833333,NaN,-41.666667,NaN,...,-22.799258,25.983341,-47.459763,-11.671260,40.662505,-33.173232,-1.270681,31.868132,43.700965,-100.000000
2000-05-31,NaN,NaN,NaN,-46.153846,NaN,NaN,-23.076923,NaN,0.000000,NaN,...,-17.146102,10.241292,-50.343868,-21.579250,13.819952,-52.193081,-1.246480,-130.952381,-121.570749,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31,0.0,0.0,0.0,-2.325581,0.0,0.0,2.272727,-8.333585,-5.405405,0.0,...,-4.263746,4.341894,-35.983797,-25.767137,-10.547156,16.126805,0.469184,258.333333,269.781291,2.631579
2023-02-28,0.0,0.0,0.0,-10.714286,0.0,0.0,-12.222222,-6.493093,-18.571429,0.0,...,-6.816496,1.738756,-58.215218,-35.284025,-13.161201,11.623157,0.783950,-22.077922,-22.838478,1.831502
2023-03-31,0.0,0.0,0.0,-1.333333,0.0,0.0,11.392405,39.490094,19.298246,0.0,...,-4.465818,0.413010,-75.833781,-27.038803,-14.254101,-5.950147,1.142634,-228.333333,-219.218854,-2.158273
2023-04-30,0.0,0.0,0.0,2.702703,0.0,0.0,4.545455,15.384745,10.294118,0.0,...,-0.048177,0.319255,-22.381153,-1.814361,-13.420505,-19.268622,6.854616,84.615385,78.780613,-4.411765


In [13]:
data.y_val_tensor

tensor([0.4500])

In [8]:
fin_data["CGO_CP"]

2000-01-31    30.00
2000-02-29    44.25
2000-03-31    42.00
2000-04-30    34.50
2000-05-31    32.75
              ...  
2023-01-31    59.31
2023-02-28    60.50
2023-03-31    60.83
2023-04-30    57.67
2023-05-31    53.66
Name: CGO_CP, Length: 281, dtype: float64

In [5]:
# from helpers.iterative import _format_tensors_it
# fin_data = pd.read_parquet("./DATA/Monthly/Processed/month_data_fin_tec.parquet")
# macro_data = pd.read_parquet("./DATA/Monthly/Processed/month_data_macro_USCA.parquet")
# z = _format_tensors_it(fin_data, macro_data, ticker, lookback=12, pred_horizon=1, multistep=False, pred_target="return")

In [10]:
data = IT_DATAMODULE(batch_size=BATCH_SIZE, lookback=LOOKBACK, pred_horizon=PRED_HORIZON, multistep=False, data_type="monthly", ticker="RGD", overwrite_cache=True, pred_target="price")

In [12]:
data.prepare_data()
data.setup()
data.X_val_tensor.shape, data.y_val_tensor.shape

  0%|          | 0/269 [00:00<?, ?it/s]

(torch.Size([1, 12, 158]), torch.Size([1]))

In [ ]:
model = CNN_1D_LSTM(input_size=data.X_val_tensor.shape[-1], hidden_size=128, num_layers=2, output_size=1, dropout=0, lr=LEARING_RATE)

In [ ]:
pred = model(data.X_val_tensor).cpu().detach().numpy()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

mean_absolute_error(data.y_val_tensor, pred)

In [ ]:
early_stopping = pl.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
convlstm_checkpoint = pl.callbacks.ModelCheckpoint(save_top_k=1, monitor="val_loss", mode="min")

#compiled_model = torch.compile(model, mode="reduce-overhead", backend="aot_eager")
mps.empty_cache()

convlstm_trainer = pl.Trainer(accelerator="gpu", max_epochs=EPOCHS, log_every_n_steps=1, callbacks=[early_stopping, convlstm_checkpoint], enable_checkpointing=True, enable_progress_bar=True, default_root_dir="./lightning_logs/cnnlstm/")
convlstm_trainer.fit(model=model, datamodule=data)
mps.empty_cache()